In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount= True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!mkdir -p "Gigaword"

In [ ]:
!tar xvzf "/content/gdrive/My Drive/585/Project/Gigaword/summary.tar.gz" -C "Gigaword/"

sumdata/
sumdata/DUC2004/
sumdata/DUC2004/input.txt
sumdata/DUC2004/task1_ref3.txt
sumdata/DUC2004/task1_ref0.txt
sumdata/DUC2004/task1_ref1.txt
sumdata/DUC2004/task1_ref2.txt
sumdata/Giga/
sumdata/Giga/input.txt
sumdata/Giga/task1_ref0.txt
sumdata/DUC2003/
sumdata/DUC2003/input.txt
sumdata/DUC2003/task1_ref3.txt
sumdata/DUC2003/task1_ref0.txt
sumdata/DUC2003/task1_ref1.txt
sumdata/DUC2003/task1_ref2.txt
sumdata/train/
sumdata/train/train.title.txt.gz
sumdata/train/valid.article.filter.txt
sumdata/train/valid.title.filter.txt
sumdata/train/train.article.txt.gz


In [ ]:
!gunzip -k "/content/Gigaword/sumdata/train/train.article.txt.gz"
!gunzip -k "/content/Gigaword/sumdata/train/train.title.txt.gz"

In [ ]:
!cp "/content/gdrive/My Drive/585/Project/Gigaword/val_art.txt" "Gigaword/val_art.txt"
!cp "/content/gdrive/My Drive/585/Project/Gigaword/val_ref.txt" "Gigaword/val_ref.txt"
!cp "/content/gdrive/My Drive/585/Project/Gigaword/test_art.txt" "Gigaword/test_art.txt"
!cp "/content/gdrive/My Drive/585/Project/Gigaword/test_ref.txt" "Gigaword/test_ref.txt"
!cp "/content/gdrive/My Drive/585/Project/Gigaword/vocab.txt" "Gigaword/vocab.txt"
!mkdir -p "gdrive/My Drive/585/Project/saved_models"

In [ ]:
!pip install rouge

In [ ]:
import os
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader
from rouge import Rouge



In [ ]:
#Config
class config:
  train_art_path = "Gigaword/sumdata/train/train.article.txt"
  train_ref_path = "Gigaword/sumdata/train/train.title.txt"
  val_art_path = "Gigaword/val_art.txt"
  val_ref_path = "Gigaword/val_ref.txt"
  test_art_path = "Gigaword/test_art.txt"
  test_ref_path = "Gigaword/test_ref.txt"
  vocab_path = "Gigaword/vocab.txt"
  model_save_path = "gdrive/My Drive/585/Project/saved_models"
  
  art_max_len = 75
  ref_max_len = 30
  batch_size = 64
  n_hid = 512
  beam_size = 4
  n_itr = 30000
  lr = 0.001
 
  use_attn = True
  sampling_type = "greedy"
  #beam_size = 4
  
  rand_unif_init_mag = 0.02
  trunc_norm_init_std = 1e-4

In [ ]:

config.bos = "<s>"
config.eos = "</s>"
config.pad = "<pad>"
config.unk = "<unk>"
config.n_vocab = 30000


In [ ]:
class Vocab(object):
  def __init__(self, config, max_size):
    self.config = config
    self._word_to_id = {}
    self._id_to_word = {}
    self._count = 0 # keeps track of total number of words in the Vocab

    # [UNK], [PAD], [START] and [STOP] get the ids 0,1,2,3.
    for w in [config.unk, config.pad, config.bos, config.eos]:
      self._word_to_id[w] = self._count
      self._id_to_word[self._count] = w
      self._count += 1

    # Read the vocab file and add words up to max_size
    with open(config.vocab_path, 'r') as vocab_f:
      for line in vocab_f:
        pieces = line.split()
        if len(pieces) != 2:
          continue
        w = pieces[0]
        self._word_to_id[w] = self._count
        self._id_to_word[self._count] = w
        self._count += 1
        if max_size != 0 and self._count >= max_size:
          # print ("max_size of vocab was specified as %i; we now have %i words. Stopping reading." % (max_size, self._count))
          break
    
  def word2id(self, word):
    if word not in self._word_to_id:
      return self._word_to_id[self.config.unk]
    return self._word_to_id[word]

  def id2word(self, word_id):
    return self._id_to_word[word_id]
    
config.vocab_obj = Vocab(config, config.n_vocab)

In [ ]:
class Gigaword_dataset(Dataset):
  def __init__(self, art_path, ref_path, config):
    self.config = config
    self.art_max_len = config.art_max_len
    self.ref_max_len = config.ref_max_len
    art_itr = open(art_path, "r")
    ref_itr = open(ref_path, "r")
    self.pair_list = []
    for art in art_itr:
      art = art.strip()
      ref = next(ref_itr).strip()
      if len(art) < 10 or len(ref) < 5:
        continue
      self.pair_list.append((art, ref))
    art_itr.close()
    ref_itr.close()
    
  def __len__(self):
    return len(self.pair_list)
  
  def __getitem__(self,ind):
    art, ref = self.pair_list[ind]
    art_ids, seq_len, spl_mask = self.tokenize_art(art)
    ref_ids = self.tokenize_ref(ref)
    art_ids = T.LongTensor(art_ids)
    ref_ids = T.LongTensor(ref_ids)
    spl_mask = T.LongTensor(spl_mask)
    return art_ids, seq_len, ref_ids, spl_mask
  
  def tokenize_art(self, art):
    tokens = art.split(" ")
    art_ids = []
    for token in tokens:
      art_ids.append(self.config.vocab_obj.word2id(token))
    spl_token_mask = [1]*len(art_ids)
    seq_len = len(art_ids)
    return art_ids, seq_len, spl_token_mask
  
  def tokenize_ref(self, ref):
    tokens = ref.split(" ")
    tokens = tokens[:(self.ref_max_len-2)] #Considering <s> and </s>
    tokens = [self.config.bos] + tokens + [self.config.eos]
    ref_ids = []
    for token in tokens:
      ref_ids.append(self.config.vocab_obj.word2id(token))
    return ref_ids

def collate_fun(batch):
  batch_art_ids = [obj[0] for obj in batch]
  batch_seq_len = T.LongTensor([obj[1] for obj in batch])
  batch_ref_ids = [obj[2] for obj in batch]
  batch_spl_mask = [obj[3] for obj in batch]
  batch_art_ids = nn.utils.rnn.pad_sequence(batch_art_ids, batch_first=True, padding_value=config.vocab_obj.word2id(config.pad))
  batch_ref_ids = nn.utils.rnn.pad_sequence(batch_ref_ids, batch_first=True, padding_value=config.vocab_obj.word2id(config.pad))
  batch_spl_mask = nn.utils.rnn.pad_sequence(batch_spl_mask, batch_first=True, padding_value=0)
  return batch_art_ids, batch_seq_len, batch_ref_ids, batch_spl_mask

def get_train_valid_dataloaders(config):
  train_dataset = Gigaword_dataset(config.train_art_path, config.train_ref_path, config)
  train_sampler = RandomSampler(train_dataset)
  train_dataloader = DataLoader(train_dataset, batch_size = config.batch_size, 
                                sampler = train_sampler, num_workers=2,
                                collate_fn = collate_fun, drop_last = False
                               )
  
  val_dataset = Gigaword_dataset(config.val_art_path, config.val_ref_path, config)
  val_sampler = SequentialSampler(val_dataset)
  val_dataloader = DataLoader(val_dataset, batch_size = config.batch_size, 
                                sampler = val_sampler, num_workers=2,
                                collate_fn = collate_fun, drop_last = False
                               )
  return train_dataloader, val_dataloader

def get_test_dataloader(config):
  test_dataset = Gigaword_dataset(config.test_art_path, config.test_ref_path, config)
  test_sampler = SequentialSampler(test_dataset)
  test_dataloader = DataLoader(test_dataset, batch_size = config.batch_size, 
                                sampler = test_sampler, num_workers=2,
                                collate_fn = collate_fun, drop_last = False
                               )
  return test_dataloader

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTM_Encoder(nn.Module):
  def __init__(self, config):
    super(LSTM_Encoder, self).__init__()
    self.lstm = nn.LSTM(config.n_hid, config.n_hid, num_layers=1, batch_first=True, bidirectional=True)
    self.reduce_h = nn.Linear(config.n_hid * 2, config.n_hid)
    self.reduce_c = nn.Linear(config.n_hid * 2, config.n_hid)
    self.embeds = nn.Embedding(config.n_vocab, config.n_hid, padding_idx = config.vocab_obj.word2id(config.pad))
  
  def forward(self, batch_art_ids, batch_seq_len):
    embeds = self.embeds(batch_art_ids)
    packed = pack_padded_sequence(embeds, batch_seq_len, batch_first=True, enforce_sorted=False)
    enc_out, enc_hid = self.lstm(packed)
    enc_out,_ = pad_packed_sequence(enc_out, batch_first=True)
    enc_out = enc_out.contiguous()                              #bs, n_seq, 2*n_hid
    h, c = enc_hid                                              #shape of h: 2, bs, n_hid
    h = T.cat(list(h), dim=1)                                   #bs, 2*n_hid
    c = T.cat(list(c), dim=1)
    h_reduced = F.relu(self.reduce_h(h))                        #bs,n_hid
    c_reduced = F.relu(self.reduce_c(c))
    return enc_out, (h_reduced.unsqueeze(0), c_reduced.unsqueeze(0))
  

class Encoder_Attention(nn.Module):
  #Perform additive attention: scores = v^T([W_e.enc_h+W_d.dec_h]))
  def __init__(self, config):
    super(Encoder_Attention, self).__init__()
    self.W_e = nn.Linear(2*config.n_hid, config.n_hid)
    self.W_d = nn.Linear(config.n_hid, config.n_hid)
    self.v = nn.Linear(config.n_hid, 1)
  
  def forward(self, enc_h, dec_h, batch_spl_mask):
    enc_h = self.W_e(enc_h) #bs, enc_seq_len, n_hid
    dec_h = self.W_d(dec_h) #bs, dec_seq_len, n_hid
    scores = self.v(enc_h.unsqueeze(1) + dec_h.unsqueeze(2)).squeeze(3) #bs, dec_seq_len, enc_seq_len
    bs, enc_seq_len = batch_spl_mask.size()
    dec_seq_len = dec_h.size(1)
    enc_attn_mask = batch_spl_mask.unsqueeze(1).expand(bs,dec_seq_len,enc_seq_len)
    scores.masked_fill_(enc_attn_mask == 0, -1e9) #Masks [CLS], [SEP], [PAD] tokens
    attn_w = F.softmax(scores, dim=2) #bs, dec_seq_len, enc_seq_len
    context_vecs = T.bmm(attn_w, enc_h) #bs,dec_seq_len,n_hid
    return context_vecs
  

  
class LSTM_Decoder(nn.Module):
  def __init__(self, config):
    super(LSTM_Decoder, self).__init__()
    self.use_attn = config.use_attn
    self.lstm = nn.LSTM(config.n_hid, config.n_hid, batch_first=True)
    if self.use_attn:
      self.fc = nn.Linear(2*config.n_hid, config.n_vocab)
      self.attn_model = Encoder_Attention(config)
    else:
      self.fc = nn.Linear(config.n_hid, config.n_vocab)
    
  
  def forward(self, dec_embeds, enc_word_reps, batch_spl_mask, dec_hidden):
    dec_out, dec_hidden = self.lstm(dec_embeds, dec_hidden) # dec_out:- (bs,ref_n_seq-1,n_hid)
    if self.use_attn:
      context_vec = self.attn_model(enc_word_reps, dec_out, batch_spl_mask) #bs,ref_n_seq-1,n_hid
      logits = self.fc(T.cat([dec_out, context_vec], dim=2))
    else:
      logits = self.fc(dec_out) #bs,ref_n_seq-1,n_vocab
    return logits, dec_hidden


In [ ]:
class Beam(object):
  def __init__(self, config, enc_sent_rep):
    h, c = enc_sent_rep
    self.pad_id = config.vocab_obj.word2id(config.pad)
    self.bos_id = config.vocab_obj.word2id(config.bos)
    self.eos_id = config.vocab_obj.word2id(config.eos)
    self.generated_tokens = T.LongTensor(config.beam_size,1).fill_(self.bos_id).to(0)
    self.scores = T.FloatTensor(config.beam_size,1).fill_(-1e9).to(0)
    self.scores[0][0] = 0
    self.hid_h = h.repeat(1,config.beam_size,1) #1,beam_size,n_hid
    self.hid_c = c.repeat(1,config.beam_size,1)
    self.n_vocab = config.n_vocab
    self.beam_size = config.beam_size
    self.done = False
    
  def get_prev_tokens(self):
    return self.generated_tokens[:,-1].unsqueeze(1) #beam,1
  
  def advance(self, log_probs, hidden_vecs):
    h, c = hidden_vecs #beam, n_hid each
    scores = log_probs + self.scores #beam, n_vocab
    scores = scores.view(-1,1) #beam*n_vocab,1
    best_scores, best_score_ids = T.topk(input=scores, k=self.beam_size,dim=0)
    self.scores = best_scores #(beam,1) sorted
    best_beam_inds = best_score_ids.squeeze(1)//self.n_vocab #(beam,) sorted
    best_tokens = best_score_ids%self.n_vocab #(beam,1)
    self.hid_h = h[best_beam_inds].unsqueeze(0) #1,beam,n_hid
    self.hid_c = c[best_beam_inds].unsqueeze(0) #1,beam,n_hid
    self.generated_tokens = self.generated_tokens[best_beam_inds] #beam,t-1
    self.generated_tokens = T.cat([self.generated_tokens, best_tokens], dim=1) #beam,t
    
    #End condition is when top-of-beam is EOS.
    if best_tokens[0].item() == self.eos_id:
      self.done = True
    
  def get_best(self):
    return self.generated_tokens[0]
  
      

In [ ]:
class Lstm_Lstm_Processor(object):
  def __init__(self, config, mode = "train"):
    self.n_vocab = config.n_vocab
    self.encoder = LSTM_Encoder(config).to(0)
    self.decoder = LSTM_Decoder(config).to(0)
    if mode == "train":
      train_dataloader, val_dataloader = get_train_valid_dataloaders(config)
      self.train_dataloader, self.val_dataloader = train_dataloader, val_dataloader
      self.enc_optimizer = T.optim.Adam(self.encoder.parameters(), lr = config.lr)
      self.dec_optimizer = T.optim.Adam(self.decoder.parameters(), lr = config.lr)
      self.encoder.train()
      self.decoder.train()
    else:
      self.test_dataloader = get_test_dataloader(config)
      self.encoder.eval()
      self.decoder.eval()
      
    self.vocab_obj = config.vocab_obj
    self.config = config
    self.pad_id = self.vocab_obj.word2id(config.pad)
    self.bos_id = self.vocab_obj.word2id(config.bos)
    self.eos_id = self.vocab_obj.word2id(config.eos)
    
  def train_batch(self, batch):
    batch_art_ids, batch_seq_len, batch_ref_ids, batch_spl_mask = batch
    batch_art_ids = batch_art_ids.to(0) #bs, art_n_seq
    batch_ref_ids = batch_ref_ids.to(0) #bs, ref_n_seq
    batch_spl_mask = batch_spl_mask.to(0) #bs, art_n_seq
    #------Encoding----------
    enc_word_reps, enc_sent_rep = self.encoder(batch_art_ids, batch_seq_len)
    #------Decoding----------
    dec_inp = batch_ref_ids[:, :-1] #ignoring </s>; bs,ref_n_seq-1
    dec_target = batch_ref_ids[:, 1:] #ignoring <s>; bs,ref_n_seq-1
    dec_hidden = enc_sent_rep
    dec_embeds = self.encoder.embeds(dec_inp) #bs,ref_n_seq-1,n_hid
    dec_logits, _ = self.decoder(dec_embeds, enc_word_reps, batch_spl_mask, dec_hidden)
    #----- Training-----------
    dec_target = dec_target.contiguous().view(-1) #(bs*(ref_n_seq-1),)
    dec_logits = dec_logits.view(-1,self.n_vocab) #bs*(ref_n_seq-1),n_vocab
    
    loss = F.cross_entropy(dec_logits, dec_target, ignore_index=self.pad_id)
    
    self.enc_optimizer.zero_grad()
    self.dec_optimizer.zero_grad()
    loss.backward()
    self.enc_optimizer.step()
    self.dec_optimizer.step()
    
    return loss.item()
  
  def evaluate_rouge(self):
    #ToDo Evaluate using Rouge
    self.encoder.eval()
    self.decoder.eval()
    rouge = Rouge()
    #------------------ 
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    for batch in self.val_dataloader:
      batch_art_ids, batch_seq_len, batch_ref_ids, batch_spl_mask = batch
      batch_art_ids = batch_art_ids.to(0) #bs, art_n_seq
      batch_spl_mask = batch_spl_mask.to(0) #bs, art_n_seq
      with T.autograd.no_grad():
        enc_word_reps, enc_sent_rep = self.encoder(batch_art_ids, batch_seq_len)
        dec_hidden = enc_sent_rep
        if self.config.sampling_type == "greedy":
          generated_tokens = self.generate_samples_greedy(enc_word_reps, batch_spl_mask, dec_hidden)
        elif self.config.sampling_type == "beamsearch":
          generated_tokens = self.generate_samples_beamsearch(enc_word_reps, batch_spl_mask, dec_hidden)
      ref_dec = self.convert_ids_to_strings(generated_tokens, type_="ref")
      ref_gold = self.convert_ids_to_strings(batch_ref_ids, type_="ref")
      batch_rouge = rouge.get_scores(ref_gold, ref_dec, avg=True)
      rouge1_scores.append(batch_rouge["rouge-1"]["f"])
      rouge2_scores.append(batch_rouge["rouge-2"]["f"])
      rougeL_scores.append(batch_rouge["rouge-l"]["f"])
    #------------------
    self.encoder.train()
    self.decoder.train()
    return np.mean(rouge1_scores), np.mean(rouge2_scores), np.mean(rougeL_scores)
  
  def generate_samples_beamsearch(self, enc_word_reps, batch_spl_mask, dec_hidden):
    batch_size, art_n_seq, n_hid = enc_word_reps.size()
    beam_idx = T.LongTensor(list(range(batch_size)))
    beams = [Beam(self.config, (dec_hidden[0][:,i,:], dec_hidden[1][:,i,:])) for i in range(batch_size)]
    n_rem = batch_size
    
    for _ in range(self.config.ref_max_len):
      prev_tokens = T.cat(
          [beam.get_prev_tokens() for beam in beams if beam.done == False], dim=0
      ) #rem*beam,1
      dec_embeds = self.encoder.embeds(prev_tokens)
      
      dec_h = T.cat(
          [beam.hid_h for beam in beams if beam.done == False], dim=1
      ) #1,rem*beam,n_hid
      dec_c = T.cat(
          [beam.hid_c for beam in beams if beam.done == False], dim=1
      ) #1,rem*beam,n_hid
      dec_hidden = (dec_h, dec_c)
      
      enc_word_reps_beam = enc_word_reps[beam_idx].view(n_rem,-1).repeat(1,config.beam_size).view(-1, art_n_seq, n_hid)
      batch_spl_mask_beam = batch_spl_mask[beam_idx].repeat(1,config.beam_size).view(-1, art_n_seq)
      
      dec_logits, (dec_h, dec_c) = self.decoder(dec_embeds, enc_word_reps_beam, batch_spl_mask_beam, dec_hidden)
      log_probs = F.log_softmax(dec_logits, dim=2)
      
      log_probs = log_probs.view(n_rem, self.config.beam_size, self.config.n_vocab) #rem, beam, n_vocab
      dec_h = dec_h.view(n_rem, config.beam_size, self.config.n_hid) #rem, beam, n_hid
      dec_c = dec_c.view(n_rem, config.beam_size, self.config.n_hid) #rem, beam, n_hid
      
      active = []
      for i in range(n_rem):
        b = beam_idx[i].item()
        beam = beams[b]
        beam.advance(log_probs[i], (dec_h[i], dec_c[i]))
        if beam.done == False:
          active.append(b)

      if len(active) == 0:
        break

      beam_idx = T.LongTensor(active)
      n_rem = len(beam_idx)
    
    generated_tokens = []
    for beam in beams:
      generated_tokens.append(beam.get_best())
    generated_tokens = nn.utils.rnn.pad_sequence(generated_tokens, batch_first=True, padding_value=self.vocab_obj.word2id(self.config.pad))
    return generated_tokens
      
  def generate_samples_greedy(self, enc_word_reps, batch_spl_mask, dec_hidden):
    #ToDo: Generate using Beam Search
    bs = len(enc_word_reps)
    cur_token = T.LongTensor(bs,1).fill_(self.bos_id).to(0)
    generated_tokens = [cur_token]
    for t in range(1,self.config.ref_max_len):
      dec_embeds = self.encoder.embeds(cur_token)
      dec_logits, dec_hidden = self.decoder(dec_embeds, enc_word_reps, batch_spl_mask, dec_hidden) #bs,1,n_vocab
      word_dist = F.softmax(dec_logits.squeeze(1), dim=1) #bs,n_vocab
      cur_token = T.multinomial(word_dist, 1)
      generated_tokens.append(cur_token)
    generated_tokens = T.cat(generated_tokens, dim=1) #bs,ref_max_len
    return generated_tokens
    
  def convert_ids_to_strings(self, batch_ids, type_):
      batch_ids = batch_ids.cpu().numpy()
      strs = []
      if type_ == "article":
        end_id = self.pad_id
      else:
        end_id = self.eos_id
      for i in range(len(batch_ids)):
        sent_ids = batch_ids[i]
        eos_tokens = np.where(sent_ids == end_id)
        if len(eos_tokens[0]) > 0:
          stop_token = eos_tokens[0][0]
        else:
          pad_tokens = np.where(sent_ids == self.pad_id)
          if len(pad_tokens[0]) > 0:
            stop_token = pad_tokens[0][0]
          else:
            stop_token = len(sent_ids)
        sent_ids = sent_ids[1:stop_token] #Exclude cls, bos, sep, eos, pad tokens
        if len(sent_ids) < 3:
          sent_string = "xxx"
        else:
          sent_string = []
          for id in sent_ids:
            sent_string.append(self.vocab_obj.id2word(id))
          sent_string = " ".join(sent_string)
        strs.append(sent_string)
      return strs
     
  def save_model(self, itr, loss, final=False):
    if final:
      save_name = "final.tar"
    else:
      save_name = "%06d.tar"%(itr)
    save_path = os.path.join(self.config.model_save_path, save_name)
    T.save({
        "itr": itr,
        "loss": loss,
        "encoder":self.encoder.state_dict(),
        "decoder":self.decoder.state_dict(),
        "enc_optimizer": self.enc_optimizer.state_dict(),
        "dec_optimizer": self.dec_optimizer.state_dict()
    }, save_path)
    
  def load_model(self, model_name, mode = "train"):
    checkpoint = T.load(os.path.join(self.config.model_save_path, model_name))
    self.encoder.load_state_dict(checkpoint["encoder"])
    self.decoder.load_state_dict(checkpoint["decoder"])
    if mode == "train":
      self.enc_optimizer.load_state_dict(checkpoint["enc_optimizer"])
      self.dec_optimizer.load_state_dict(checkpoint["dec_optimizer"])
    return checkpoint["itr"], checkpoint["loss"]
  


In [ ]:
def training(model_name = None):
  loss_arr = []
  r1_arr = []
  r2_arr = []
  rL_arr = []
  train_processor = Lstm_Lstm_Processor(config)
  if model_name is None:
    itr = 1
  else:
    itr, prev_loss = train_processor.load_model(model_name)
    itr += 1
    print("Loaded Model at:",itr, "; loss:", prev_loss)
  while(itr<=config.n_itr):
    for batch in train_processor.train_dataloader:
      try:
        loss = train_processor.train_batch(batch)
      except KeyboardInterrupt:
        exit(0)
      loss_arr.append(loss)
      if itr%100 == 0:
        print("itr:", itr, "loss:", "%.3f"%(loss))
      if itr%1500 == 0:
        rouge1, rouge2, rougeL = train_processor.evaluate_rouge()
        r1_arr.append(rouge1)
        r2_arr.append(rouge2)
        rL_arr.append(rougeL)
        print("val scores:-", "Rouge-1:","%.3f"%(rouge1), "Rouge-2:","%.3f"%(rouge2), "Rouge-L:","%.3f"%(rougeL))
      if itr%5000 == 0:
        train_processor.save_model(itr, loss, final=True)
      if itr == config.n_itr:
        print("Reached")
        T.save({
              "loss_arr" : loss_arr,
              "r1_arr" : r1_arr,
              "r2_arr" : r2_arr,
              "rL_arr" : rL_arr,
          }, "/content/gdrive/My Drive/585/scores.tar")
        print("Saved")
      itr += 1
      if itr > config.n_itr:
        break
        
def testing(model_name):
  rouge = Rouge()
  train_processor = Lstm_Lstm_Processor(config, mode="test")
  itr, prev_loss = train_processor.load_model(model_name, mode = "test")
  print("Loaded Model at:",itr, "; loss:", prev_loss)
  for batch in train_processor.test_dataloader:
    batch_art_ids, batch_seq_len, batch_ref_ids, batch_spl_mask = batch
    batch_art_ids = batch_art_ids.to(0) #bs, art_n_seq
    batch_spl_mask = batch_spl_mask.to(0) #bs, art_n_seq
    
    with T.autograd.no_grad():
      enc_word_reps, enc_sent_rep = train_processor.encoder(batch_art_ids, batch_seq_len)
      dec_hidden = enc_sent_rep
      if train_processor.config.sampling_type == "greedy":
        generated_tokens = train_processor.generate_samples_greedy(enc_word_reps, batch_spl_mask, dec_hidden)
      elif train_processor.config.sampling_type == "beamsearch":
        generated_tokens = train_processor.generate_samples_beamsearch(enc_word_reps, batch_spl_mask, dec_hidden)
    ref_dec_strs = train_processor.convert_ids_to_strings(generated_tokens, type_="ref")
    article_strs = train_processor.convert_ids_to_strings(batch_art_ids, type_="article")
    ref_gold_strs = train_processor.convert_ids_to_strings(batch_ref_ids, type_="ref")
    for article, ref_dec, ref_gold in zip(article_strs, ref_dec_strs, ref_gold_strs):
      print("Article:", article)
      print("Ref_dec:", ref_dec)
      print("Ref_gold:", ref_gold)
      scores = rouge.get_scores(ref_gold, ref_dec, avg=True)
      print("Rouge:", scores["rouge-l"]["f"])
      print("--------------------------")
    break
    
#training()
testing('final.tar')
  

Loaded Model at: 30000 ; loss: 3.3685338497161865
Article: activists from throughout the west plan to converge on northern california town of <unk> sunday for the third annual rally to save <unk> forest .
Ref_dec: upper house maathai _ in straight west event failing singers west
Ref_gold: environmental groups plan protest at forest of <unk>
Rouge: 0.0
--------------------------
Article: an arab world allergic to political reform , the new iraq taking shape under u.s. <unk> is a troubling harbinger .
Ref_dec: arab world bank promotes assessment of iraqi corruption
Ref_gold: to <unk> arab rulers new iraq is an unwanted role model
Rouge: 0.1003798155174875
--------------------------
Article: 's a car that answers questions that nobody was asking , a vehicle that looks as if it were the work of the of <unk> or disney studios rather than an exercise in automotive design .
Ref_dec: james astronomers proves what they design the escalation
Ref_gold: suzuki 's <unk> mini poses a major question 

In [ ]:

checkpoint = T.load("/content/gdrive/My Drive/585/scores.tar")
r1_arr = checkpoint['r1_arr']
r2_arr = checkpoint['r2_arr']
rL_arr = checkpoint['rL_arr']
print(r1_arr)
print(r2_arr)
print(rL_arr)



[0.13621861659302445, 0.16473979876362838, 0.18061692359923553, 0.1882501750881013, 0.20061959359603773, 0.20510242863629807, 0.20455946265891164, 0.21113906247058623, 0.21272430555160265, 0.21368561235318276, 0.21485634182413577, 0.21691521279985204, 0.2170823799633546, 0.21997096424093637, 0.22092137285084626, 0.22883802465554742, 0.2280097737350058, 0.2267971646173757, 0.22373036149928532, 0.22956357959349213]
[0.029169209267163185, 0.03952115948404685, 0.04502421233379681, 0.04919413955886631, 0.05387434826295175, 0.055499176667447615, 0.05541699188861595, 0.05705860236964505, 0.05742737568478614, 0.05967013287409832, 0.05917406764442135, 0.058795905986917946, 0.06017710124766253, 0.061028479943143095, 0.061538128533290784, 0.06555909150105699, 0.0653074998071326, 0.06551922877875167, 0.06284572452046142, 0.06679438939162441]
[0.12254772996923641, 0.14737655693368631, 0.16186077479085428, 0.16786310096422552, 0.17913072381495598, 0.18305395982991507, 0.18262049144133208, 0.18847666